In [1]:
from datetime import datetime

import time
import os
import sys
from pathlib import Path

import numpy as np
from scipy.stats import lognorm
import pandas as pd

from astropy import stats
from astropy.io import fits
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.utils import iers
import astropy.units as u
import astroplan

import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
iers.IERS_A_URL = 'ftp://cddis.gsfc.nasa.gov/pub/products/iers/finals2000A.all'
astroplan.download_IERS_A()

In [4]:
%cd /home/halcoll/halcoll/halcoll/data

/home/halcoll/halcoll/halcoll/data


In [5]:
years = []
for y in [2016, 2017, 2018, 2019]:
    years.append(pd.read_csv(f"{y}_e_series_trimmed.csv"))
temp_data = pd.concat(years, sort=True)

In [33]:
lat = "31:41:19.6"
lon = "-110:53:04.4"
elevation = 2600 * u.m
location = EarthLocation.from_geodetic(lon, lat, elevation)
mmt = astroplan.Observer(name="MMTO", location=location, timezone="US/Arizona", pressure=0*u.mbar)
tset = mmt.sun_set_time(Time(datetime.now()), which='next', horizon=-0.8333*u.deg)
tset.isot

'2019-11-16T00:24:21.756'

In [9]:
from skyfield import api
ts = api.load.timescale()
e = api.load('de421.bsp')
from skyfield import almanac

[#################################] 100% deltat.data
[#################################] 100% deltat.preds
[#################################] 100% Leap_Second.dat
[#################################] 100% de421.bsp


In [29]:
mmt_sf = api.Topos(latitude_degrees=location.lat.value, longitude_degrees=location.lon.value, elevation_m=2600.0)

In [34]:
t0 = ts.utc(2019, 11, 15, 23)
t1 = ts.utc(2019, 11, 26, 23)
t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, mmt_sf))
print(t.utc_iso())
print(y)

['2019-11-16T00:24:23Z', '2019-11-16T13:52:25Z', '2019-11-17T00:23:52Z', '2019-11-17T13:53:17Z', '2019-11-18T00:23:23Z', '2019-11-18T13:54:10Z', '2019-11-19T00:22:55Z', '2019-11-19T13:55:02Z', '2019-11-20T00:22:29Z', '2019-11-20T13:55:55Z', '2019-11-21T00:22:05Z', '2019-11-21T13:56:47Z', '2019-11-22T00:21:43Z', '2019-11-22T13:57:40Z', '2019-11-23T00:21:22Z', '2019-11-23T13:58:32Z', '2019-11-24T00:21:03Z', '2019-11-24T13:59:24Z', '2019-11-25T00:20:46Z', '2019-11-25T14:00:16Z', '2019-11-26T00:20:30Z', '2019-11-26T14:01:08Z']
[False  True False  True False  True False  True False  True False  True
 False  True False  True False  True False  True False  True]


In [35]:
almanac.find_discrete?

Signature:
almanac.find_discrete(
    start_time,
    end_time,
    f,
    epsilon=1.1574074074074074e-08,
    num=12,
)
Docstring:
Find the times when a function changes value.

Searches between ``start_time`` and ``end_time``, which should both
be :class:`~skyfield.timelib.Time` objects, for the occasions where
the function ``f`` changes from one value to another.  Use this to
search for events like sunrise or moon phases.

A tuple of two arrays is returned. The first array gives the times
at which the input function changes, and the second array specifies
the new value of the function at each corresponding time.

This is an expensive operation as it needs to repeatedly call the
function to narrow down the times that it changes.  It continues
searching until it knows each time to at least an accuracy of
``epsilon`` Julian days.  At each step, it creates an array of
``num`` new points between the lower and upper bound that it has
established for each transition.  These two values can 

In [37]:
y

array([False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True])